# 5 | Data Exports for Tableau Dashboard

This project produces several CSV files that feed directly into the Tableau dashboard. Each file supports a specific visualization component, as described below:

| File                                      | Purpose (Tableau Component)                      |
| ----------------------------------------- | ------------------------------------------------ |
| **business_churn_by_category.csv**        | Bar / Pie Chart – Number and Share of Churned Businesses by Primary Category |
| **state_business_churn.csv**              | Choropleth Map – Business Churn Rate by State    |
| **business_churn_by_open_year.csv**       | Line / Column Chart – Opening Year vs Churn Rate |
| **business_churn_timeline.csv**           | Heatmap (Year‑Month) – Distribution of Last Review Dates |
| **user_activity_retention.csv**           | Bar Chart – User Activity vs. Retention          |
| **user_churn_by_state.csv**               | Choropleth Map – User Churn Rate by State        |



In [1]:
# 0 | Imports & Paths
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

RAW  = Path("../data/raw")
PROC = Path("../data/processed")
PROC.mkdir(exist_ok=True, parents=True)

In [3]:
# 1 | Load business churn profile
biz_profile = pd.read_csv(PROC / "business_churn_profile.csv",
                          parse_dates=["date_first", "date_last"])
print("Rows:", len(biz_profile))
biz_profile.head()

Rows: 150346


,business_id,name,city,state,categories,stars,review_count,is_open,latitude,longitude,date_first,date_last,cnt_hist,cnt_recent,star_hist,star_recent,checkins_total,no_review_recent,rating_drop,churn_flag
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,CA,"Doctors, Traditional Chinese Medicine, Naturop...",5.0,7,0,34.426679,-119.711197,2012-05-02 18:07:38,2015-03-16 03:43:08,7,0,4.857143,NaN,1.0,True,False,False
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,"Shipping Centers, Local Services, Notaries, Ma...",3.0,15,1,38.551126,-90.335695,2011-08-01 03:45:56,2022-01-19 18:57:45,15,2,3.133333,3.000000,15.0,False,False,False
2,tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,AZ,"Department Stores, Shopping, Fashion, Home & G...",3.5,22,0,32.223236,-110.880452,2008-12-14 08:41:21,2020-12-02 02:50:27,24,0,3.500000,NaN,401.0,True,True,False
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",4.0,80,1,39.955505,-75.155564,2008-03-09 00:36:56,2021-11-01 18:22:07,87,6,4.057471,4.166667,335.0,False,False,False
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,"Brewpubs, Breweries, Food",4.5,13,1,40.338183,-75.471659,2019-07-07 22:59:25,2021-10-16 20:37:44,13,2,4.692308,5.000000,14.0,False,False,False


In [4]:
# 2 | Main category aggregation
def pick_first(cat):
    return cat.split(",")[0].strip() if isinstance(cat, str) else "Unknown"

biz_profile["main_cat"] = biz_profile["categories"].apply(pick_first)

cat_df = (
    biz_profile
      .groupby("main_cat")
      .agg(total_business=("business_id", "count"),
           churned=("churn_flag",   "sum"))
      .assign(churn_rate=lambda d: d["churned"] / d["total_business"])
      .sort_values("churned", ascending=False)
      .reset_index()
)

cat_df.to_csv(PROC/"business_churn_by_category.csv", index=False)
print("✅ business_churn_by_category.csv saved")
cat_df.head()

✅ business_churn_by_category.csv saved


,main_cat,total_business,churned,churn_rate
0,Restaurants,15290,808,0.052845
1,Beauty & Spas,4385,607,0.138426
2,Shopping,5480,596,0.108759
3,Food,6783,529,0.077989
4,Health & Medical,3058,376,0.122956


In [5]:
# 3 | State-level business churn
state_df = (
    biz_profile
      .groupby("state")
      .agg(total_business=("business_id","count"),
           churned=("churn_flag","sum"))
      .assign(churn_rate=lambda d: d["churned"]/d["total_business"])
      .reset_index()
)
state_df.to_csv(PROC/"state_business_churn.csv", index=False)
print("✅ state_business_churn.csv saved")
state_df.sort_values("churn_rate", ascending=False).head()

✅ state_business_churn.csv saved


,state,total_business,churned,churn_rate
24,VT,1,1,1.000000
22,UT,1,1,1.000000
6,HI,2,1,0.500000
11,MA,2,1,0.500000
0,AB,5573,1134,0.203481


In [6]:
# 4 | Business churn by open year
year_df = (
    biz_profile
      .assign(open_year=biz_profile["date_first"].dt.year)
      .groupby("open_year")
      .agg(total=("business_id","count"),
           churned=("churn_flag","sum"))
      .assign(churn_rate=lambda d: d["churned"]/d["total"])
      .reset_index()
      .sort_values("open_year")
)
year_df.to_csv(PROC/"business_churn_by_open_year.csv", index=False)
print("✅ business_churn_by_open_year.csv saved")
year_df.tail()

✅ business_churn_by_open_year.csv saved


,open_year,total,churned,churn_rate
13,2018,7679,306,0.039849
14,2019,6145,110,0.017901
15,2020,3687,8,0.002170
16,2021,2282,0,0.000000
17,2022,20,0,0.000000


In [7]:
# 5 | Timeline heat-map matrix (年 × 月)
timeline = (
    biz_profile[biz_profile["churn_flag"]]
      .assign(year=biz_profile["date_last"].dt.year,
              month=biz_profile["date_last"].dt.month)
      .groupby(["year","month"])["business_id"]
      .count()
      .reset_index(name="churned_count")
)

# Fill in missing months → for easier Tableau heatmap visualization
idx = pd.MultiIndex.from_product(
    [range(timeline["year"].min(), timeline["year"].max()+1),
     range(1,13)],
    names=["year","month"]
)
timeline = timeline.set_index(["year","month"]).reindex(idx, fill_value=0).reset_index()

timeline.to_csv(PROC/"business_churn_timeline.csv", index=False)
print("✅ business_churn_timeline.csv saved")
timeline.head()


✅ business_churn_timeline.csv saved


,year,month,churned_count
0,2011,1,0
1,2011,2,0
2,2011,3,0
3,2011,4,0
4,2011,5,1


In [13]:
# 6-A | Load user features
user_feat = pd.read_csv(PROC / "user_churn_features.csv",
                        parse_dates=["last_review_date"])
print("Users:", len(user_feat))

# 6-B | Segment activity levels → Retention rates
bins = [0, 5, 20, 50, 1e9]
labels = ["Low (≤5 reviews)", "Medium (6–20)", "High (21–50)", "Very High (>50)"]
user_feat["act_bin"] = pd.cut(user_feat["review_count"], bins=bins, labels=labels, right=True)
act_ret = (
    user_feat
      .groupby("act_bin")
      .agg(users=("user_id","count"),
           churned=("churn_label","sum"))
      .assign(retention=lambda d: 1 - d["churned"]/d["users"])
      .reset_index()
)

act_ret.to_csv(PROC/"user_activity_retention.csv", index=False)
print("✅ user_activity_retention.csv saved")
act_ret

Users: 1987929
✅ user_activity_retention.csv saved


C:\Users\Haonan\AppData\Local\Temp\ipykernel_26700\4222866853.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("act_bin")


,act_bin,users,churned,retention
0,Low (≤5 reviews),1002990,849091,0.153440
1,Medium (6–20),565506,474814,0.160373
2,High (21–50),233467,195024,0.164661
3,Very High (>50),185880,151901,0.182801


In [12]:
# 6-C | Infer user's last review state (Optional – for map use)
# Helper: business_id → state dictionary
bid2state = dict(biz_profile[["business_id","state"]].values)

user2last_bid = {}     # {user_id: (last_date, last_bid)}

reader_u = pd.read_json(
    RAW/"yelp_academic_dataset_review.json",
    lines=True, chunksize=250_000, encoding="utf-8",
    dtype={"user_id":"string","business_id":"string","date":"string"}
)

for chunk in tqdm(reader_u, desc="Scanning reviews for user location"):
    for uid, grp in chunk.groupby("user_id"):
        last_row = grp.loc[grp["date"].idxmax()]
        dt  = pd.to_datetime(last_row["date"])
        bid = last_row["business_id"]
        if (uid not in user2last_bid) or (dt > user2last_bid[uid][0]):
            user2last_bid[uid] = (dt, bid)

# → DataFrame
u_loc = (pd.DataFrame([
        (u, bid, bid2state.get(bid, "XX"))
        for u, (_, bid) in user2last_bid.items()
    ], columns=["user_id","business_id","state"])
)

# Merge churn labels
u_loc = u_loc.merge(user_feat[["user_id","churn_label"]], on="user_id", how="left")

state_user = (
    u_loc.groupby("state")
         .agg(total_users=("user_id","count"),
              churned=("churn_label","sum"))
         .assign(churn_rate=lambda d: d["churned"]/d["total_users"])
         .reset_index()
)

state_user.to_csv(PROC/"user_churn_by_state.csv", index=False)
print("✅ user_churn_by_state.csv saved (for user map)")
state_user.head()


Scanning reviews for user location: 28it [29:04, 62.30s/it]


✅ user_churn_by_state.csv saved (for user map)


,state,total_users,churned,churn_rate
0,AB,23240,21239,0.913898
1,AZ,118707,96813,0.815563
2,CA,145082,125613,0.865807
3,CO,18,18,1.000000
4,DE,23073,19446,0.842803


In [11]:
print("Part-5  Key summary file list:")
for f in [
    "business_churn_by_category.csv",
    "state_business_churn.csv",
    "business_churn_by_open_year.csv",
    "business_churn_timeline.csv",
    "user_activity_retention.csv",
    "user_churn_by_state.csv",
]:
    p = PROC / f
    print(f"{f:35s}  -->  {'OK' if p.exists() else 'MISSING'}")

Part-5  Key summary file list:
business_churn_by_category.csv       -->  OK
state_business_churn.csv             -->  OK
business_churn_by_open_year.csv      -->  OK
business_churn_timeline.csv          -->  OK
user_activity_retention.csv          -->  OK
user_churn_by_state.csv              -->  OK
